# Project Management

## Internals

In [3]:
import pandas as pd
import datetime as dt
import numpy as np
import plotly.plotly as py
from plotly.tools import FigureFactory as FF

### Param

In [4]:
_pathTasks='tasks.csv'
_pathAvail='avail.csv'
_ProjectStartDate = dt.datetime(year=2016,month=11,day=21)
_1DayTimeDelta = dt.timedelta(1)

### Load data

In [5]:
_TaskHeaders = ['TaskId','Name','WorkLoad','Who','StartDate','EndDate','Priority','PredecessorTaskId']

def LoadTask(path):
        data = pd.read_csv(path, header=0,names=_TaskHeaders, dtype={'PredecessorTaskId':object})
        data.StartDate = pd.to_datetime(data.StartDate)
        data.EndDate = pd.to_datetime(data.EndDate)
        data.index=data.TaskId
        return data
def LoadAvailability(path):
    '''
    This function load an availability matrix
    '''
    availCSV = pd.read_csv(path)
    availCSV.Date = pd.to_datetime(availCSV.Date)
    availCSV.sort_values(inplace=True,by='Date')
    return availCSV
    

### Save

In [6]:
def SaveData(data,path):
    data.to_csv(path)

### Domain
* Find tasks of a personn
* Order tasks by priority then ID (just in case)
* Compute end dates
    * Find previous task of person and previous task id
    * Based on the max start date and workload, iterate to see the day when the task will be finised


In [7]:
def FindPreviousTaskOfSomeone(tasks, current):
    '''
    Find the previous task in order of doing
    Expects an order tasks pandas by priority desc and taskid 
    '''
    whosTasks = FindTasksOfSomeone(tasks,current.Who)    
    prevTasks= whosTasks.loc[:current.TaskId]
    if len(prevTasks)<2 :
        return None
    else :
        return prevTasks.iloc[-2]
    
    

In [8]:
def FindTasksOfSomeone(tasks, who):
    return tasks[tasks.Who==who]
def FindTaskById(tasks, taskid):
    return tasks.loc[int(taskid)]
def OrderTaskByPriority(tasks):
    return tasks.sort_values(by=['Priority','TaskId'], ascending=[False,True])

In [9]:
def InitEmptyOccupation(avail,startDate, endDate):
    users = avail.Who.unique()
    duration = (endDate-startDate).days
    return pd.DataFrame(data=np.zeros(shape=(duration,len(users)), dtype=np.int32),\
                        columns=users,\
                        index=np.arange(startDate,endDate, dtype='datetime64[D]'))

In [10]:
def ComputeEndDate(availability, task):
    '''
    Compute the end date based on the occupation of someone, a start date and a duration
    '''    
    availWho = availability[(availability.Who==task.Who) & (availability.Date>=task.StartDate)]  
    currentOccupation = 0
    endDate=None
    for index,item in availWho.iterrows():
        currentOccupation=currentOccupation+item.Available
        #print 'Date %s & occupation %s' %(item, currentOccupation)
        if(currentOccupation==task.WorkLoad):
            endDate=item.Date
            break;
    if(endDate==None):
        raise Exception('Task cannot be ended',task)
    return endDate
        

In [11]:
def SetOccupiedBy(occupation,task):
    occupation.loc[task.StartDate:task.EndDate,task.Who]=task.TaskId

In [12]:
def OrderTasksByPriorityAndPredecessor(tasks):
    tasks = OrderTaskByPriority(tasks)
    indexTasks = tasks.TaskId.tolist()
    #print indexTasks
    '''
        While on the number of element
            If has predecessor
                Get its location 
                If bigger > cursor --> move after  (check if last item)          
            If moved : do not move cursor
            Else : next item
        '''
    lItems = len(indexTasks)
    cursor = 0
    moved = False 
    listOfMoved = []
    while (cursor<lItems):  
        taskI = tasks.loc[indexTasks[cursor]]
        moved=False
        #print '  Current task %s at position %s '%(taskI.TaskId,cursor)
        if (pd.isnull(taskI.PredecessorTaskId)==False) :
            predecIdx = indexTasks.index(int(taskI.PredecessorTaskId))    
            #print '  Predecessor %s is at position %s' %(taskI.PredecessorTaskId,predecIdx)
            if(predecIdx>cursor):
                #print '  Task %s with position %s must be moved after task %s at position %s' \
                #    %(taskI.TaskId,cursor,taskI.PredecessorTaskId,predecIdx)
                #Check if this task has already been moved, if yes we have a cycle
                if(taskI.TaskId in listOfMoved):
                    raise Exception('There is a cycle between tasks found on ',taskI.TaskId)
                listOfMoved.append(taskI.TaskId)
                indexTasks.pop(cursor)
                #print '  Popped %s' %(indexTasks)
                Moved=True
                cursor -=1
                if(predecIdx==lItems-1):
                    indexTasks.append(taskI.TaskId)
                else:
                    indexTasks.insert(predecIdx,taskI.TaskId)
        if moved==False:
                cursor +=1
        #print '  %s' %(indexTasks)
    return tasks.loc[indexTasks]

In [29]:
def ComputeOccupation(avail,tasks):
    '''
    Computer the tasks start and end date based on 
    
    1. Order of computation is based on the priority and PredecessorTaskId
        First the algorithm order by priority then check the predecessors 
    and moves each row just after its predecessor if it is before
        The algorithm will however detect incorrect cases :
    When it parcours the tasks, it checks if the PredecessorTaskId task has a end date. 
            If not, sends an error. 
            If yes uses the maximum between the previous task of the person and the predecessor
    2. By personn iterate over its tasks and compute the dates 
        2.1 If first task : prevStartDate=_projectStartDate
            else prevStartDate=previous.EndDate+1 (we cheat and say one task per day!)
        2.2 Check the predecessor end date : 
            If predecessor.EndDate = NaT --> exception (cycle)
        2.3 Take the max between both
        2.4 Compute the end date, set it and the occupation matrix
    '''
    orderedTasks = OrderTasksByPriorityAndPredecessor(tasks)
    for currtask in orderedTasks.itertuples():
        #print 'The current task is : %s' %(currtask.Index)        
        previousEndDate=_ProjectStartDate-_1DayTimeDelta
        predecessorTaskEndDate=_ProjectStartDate-_1DayTimeDelta
        
        #When did the last task finished
        previousTasks = FindPreviousTaskOfSomeone(tasks, currtask)  
        if(type(previousTasks)=='pandas.core.series.Series'):
            previousEndDate=previousTasks.EndDate
            if pd.isnull(previousEndDate):
                raise Exception('Couldn''t find the end date of the previous task',currtask.TaskId)

        #When did the predecessor finished if there is one
        if(pd.isnull(currtask.PredecessorTaskId)==False):            
            predecessorTask = FindTaskById(orderedTasks, currtask.PredecessorTaskId)
            #print '   %s %s' %(predecessorTask.TaskId, predecessorTask.EndDate)
            predecessorTaskEndDate=predecessorTask.EndDate
            if pd.isnull(predecessorTaskEndDate):
                raise Exception('Couldn''t find the end date of the predecessor task',currtask.TaskId)
        latestDate = max(predecessorTaskEndDate,previousEndDate)
        
        #The task start the next day of the previous
        orderedTasks.loc[currtask.Index,'StartDate']=latestDate+_1DayTimeDelta
        orderedTasks.loc[currtask.Index,'EndDate'] = ComputeEndDate(avail,orderedTasks.loc[currtask.Index])
        '''print 'Dates from %s to %s '%(orderedTasks.loc[currtask.Index,'StartDate'],\
                                      orderedTasks.loc[currtask.Index,'EndDate'])'''
    return orderedTasks
        

    

In [23]:
def FillAllOccupationFromTasks(tasks):
    occupation = InitEmptyOccupation(_avail,_ProjectStartDate,_ProjectStartDate+dt.timedelta(365))
    for task in tasks.itertuples():
        SetOccupiedBy(occupation,task)
    return occupation

#### Presentation

In [46]:
def GetPandaForGantChart(tasks):
    pdGant = tasks[['Name','Who','StartDate','EndDate']]
    pdGant.columns = ['Task','Resource','Start','Finish']    
    pdGant = pdGant.reset_index()
    del pdGant['TaskId']
    return pdGant

In [41]:
def DrawGant(gant):
    


## Load Data

In [31]:
_tasks = LoadTask(_pathTasks)

In [16]:
_avail = LoadAvailability(_pathAvail)

In [17]:
_tasks.head(10)

,TaskId,Name,WorkLoad,Who,StartDate,EndDate,Priority,PredecessorTaskId
TaskId,,,,,,,,
1,1,Test 1,4,CGA,NaT,NaT,150,NaN
2,2,Test 2,4,CGA,NaT,NaT,130,4
3,3,Test 3,4,ACH,NaT,NaT,140,2
4,4,Test 4,4,ACH,NaT,NaT,135,NaN


In [18]:
_avail.head(5)

,Who,Date,Available
0,CGA,2016-11-21,1
12,ACH,2016-11-21,1
13,ACH,2016-11-22,0
1,CGA,2016-11-22,1
14,ACH,2016-11-23,1


In [32]:
_tasks = ComputeOccupation(_avail,_tasks)

In [33]:
_tasks.head(10)

,TaskId,Name,WorkLoad,Who,StartDate,EndDate,Priority,PredecessorTaskId
TaskId,,,,,,,,
1,1,Test 1,4,CGA,2016-11-21,2016-11-24,150,NaN
4,4,Test 4,4,ACH,2016-11-21,2016-11-26,135,NaN
2,2,Test 2,4,CGA,2016-11-27,2016-11-30,130,4
3,3,Test 3,4,ACH,2016-12-01,2016-12-04,140,2


In [34]:
_occupation = FillAllOccupationFromTasks(_tasks)

In [47]:
_gant = GetPandaForGantChart(_tasks)

In [ ]:
ComputeOccupation(_avail,_tasks)

In [57]:
%%html
<script type="text/javascript" src="https://www.gstatic.com/charts/loader.js"></script>
       <div id="timeline" style="height: 250px;width: 2500px"></div>
   

In [59]:
%%javascript
  google.charts.load("current", {packages:["timeline"]});
  google.charts.setOnLoadCallback(drawChart);
  function drawChart() {
    var container = document.getElementById('timeline');
    var chart = new google.visualization.Timeline(container);
    var dataTable = new google.visualization.DataTable();
    dataTable.addColumn({ type: 'string', id: 'Term' });
    dataTable.addColumn({ type: 'string', id: 'Name' });
    dataTable.addColumn({ type: 'date', id: 'Start' });
    dataTable.addColumn({ type: 'date', id: 'End' });
    dataTable.addRows([
      [ '1', 'George Washington', new Date(1789, 3, 30), new Date(1797, 2, 4) ],
      [ '2', 'John Adams',        new Date(1797, 2, 4),  new Date(1801, 2, 4) ],
      [ '3', 'Thomas Jefferson',  new Date(1801, 2, 4),  new Date(1809, 2, 4) ]]);

    var options = {
      timeline: { }
    };

    chart.draw(dataTable, options);
  }

<IPython.core.display.Javascript object>

In [33]:
_occupation = InitEmptyOccupation(_avail,_ProjectStartDate,_ProjectStartDate+dt.timedelta(365))

In [44]:
len(_avail.Who.unique())

1

In [27]:
help (np.zeros)

Help on built-in function zeros in module numpy.core.multiarray:

zeros(...)
    zeros(shape, dtype=float, order='C')
    
    Return a new array of given shape and type, filled with zeros.
    
    Parameters
    ----------
    shape : int or sequence of ints
        Shape of the new array, e.g., ``(2, 3)`` or ``2``.
    dtype : data-type, optional
        The desired data-type for the array, e.g., `numpy.int8`.  Default is
        `numpy.float64`.
    order : {'C', 'F'}, optional
        Whether to store multidimensional data in C- or Fortran-contiguous
        (row- or column-wise) order in memory.
    
    Returns
    -------
    out : ndarray
        Array of zeros with the given shape, dtype, and order.
    
    See Also
    --------
    zeros_like : Return an array of zeros with shape and type of input.
    ones_like : Return an array of ones with shape and type of input.
    empty_like : Return an empty array with shape and type of input.
    ones : Return a new array setting v